In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from mlxtend.frequent_patterns import apriori, association_rules
from urllib.parse import quote_plus


In [2]:
DB = {
    "user": "root",
    "password": "ashish@8905",
    "host": "localhost",
    "port": 3306,
    "database": "retail_ana"
}

encoded_password = quote_plus(DB["password"])

engine = create_engine(
    f"mysql+pymysql://{DB['user']}:{encoded_password}@"
    f"{DB['host']}:{DB['port']}/{DB['database']}"
)

print("Database connection established")

Database connection established


In [4]:
import pandas as pd

query = "SELECT * FROM single_customer_view;"
rfm_df = pd.read_sql(query, engine)
rfm_df.head()



,customer_id,total_orders,total_quantity,total_revenue,first_purchase_ts,last_purchase_ts,customer_lifetime_days
0,12346,1,74215.0,77183.60,2011-04-12,2011-04-12,0
1,12347,7,2458.0,4310.00,2010-12-20,2011-10-31,315
2,12348,4,2341.0,1797.24,2011-02-27,2011-11-06,252
3,12349,1,631.0,1757.55,2011-08-17,2011-08-17,0
4,12350,1,197.0,334.40,2011-05-26,2011-05-26,0


In [5]:
rfm_df['last_purchase_ts'] = pd.to_datetime(rfm_df['last_purchase_ts'])

# Reference date = one day after latest purchase (Beacuse recency never be 0)
reference_date = rfm_df['last_purchase_ts'].max() + pd.Timedelta(days=1)
reference_date



Timestamp('2011-12-10 00:00:00')

In [6]:
rfm_df['Recency'] = (reference_date - rfm_df['last_purchase_ts']).dt.days
rfm_df['Frequency'] = rfm_df['total_orders']
rfm_df['Monetary'] = rfm_df['total_revenue']

rfm = rfm_df[['customer_id', 'Recency', 'Frequency', 'Monetary']]
rfm.head()


,customer_id,Recency,Frequency,Monetary
0,12346,242,1,77183.60
1,12347,40,7,4310.00
2,12348,34,4,1797.24
3,12349,115,1,1757.55
4,12350,198,1,334.40


In [7]:
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])

rfm.head()


C:\Users\ASHISH\AppData\Local\Temp\ipykernel_21376\131912844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
C:\Users\ASHISH\AppData\Local\Temp\ipykernel_21376\131912844.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
C:\Users\ASHISH\AppData\Local\Temp\ipykernel_21376\131912844.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

,customer_id,Recency,Frequency,Monetary,R_Score,F_Score,M_Score
0,12346,242,1,77183.60,1,1,5
1,12347,40,7,4310.00,3,5,5
2,12348,34,4,1797.24,3,4,4
3,12349,115,1,1757.55,2,1,4
4,12350,198,1,334.40,1,1,2


In [8]:
rfm['RFM_Score'] = (rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str))
rfm.head()


C:\Users\ASHISH\AppData\Local\Temp\ipykernel_21376\131625735.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['RFM_Score'] = (rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str))


,customer_id,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
0,12346,242,1,77183.60,1,1,5,115
1,12347,40,7,4310.00,3,5,5,355
2,12348,34,4,1797.24,3,4,4,344
3,12349,115,1,1757.55,2,1,4,214
4,12350,198,1,334.40,1,1,2,112


In [9]:
def rfm_segment(row):
    if row['R_Score'] >= 4 and row['F_Score'] >= 4 and row['M_Score'] >= 4:
        return "Champions"
    elif row['R_Score'] >= 3 and row['F_Score'] >= 3:
        return "Loyalists"
    elif row['R_Score'] <= 2 and row['F_Score'] <= 2:
        return "Hibernating"
    else:
        return "At risk"

rfm['Segment'] = rfm.apply(rfm_segment, axis=1)
rfm['Segment'].value_counts()


C:\Users\ASHISH\AppData\Local\Temp\ipykernel_21376\106867565.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['Segment'] = rfm.apply(rfm_segment, axis=1)


Segment
At risk        1190
Hibernating    1138
Loyalists      1028
Champions       982
Name: count, dtype: int64

In [10]:
rfm.head(10)

,customer_id,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment
0,12346,242,1,77183.60,1,1,5,115,Hibernating
1,12347,40,7,4310.00,3,5,5,355,Loyalists
2,12348,34,4,1797.24,3,4,4,344,Loyalists
3,12349,115,1,1757.55,2,1,4,214,Hibernating
4,12350,198,1,334.40,1,1,2,112,Hibernating
5,12352,30,8,2506.04,4,5,5,455,Champions
6,12353,9,1,89.00,5,1,1,511,At risk
7,12354,30,1,1079.40,4,1,4,414,At risk
8,12355,20,1,459.40,4,1,2,412,At risk
9,12356,46,3,2811.43,3,3,5,335,Loyalists


In [11]:
segment_validation = (
    rfm.groupby('Segment')['Monetary'].mean().sort_values(ascending=False)
)


In [12]:
# average revenue by segment
segment_validation

Segment
Champions      6049.923167
Loyalists      1396.394008
At risk         852.654126
Hibernating     457.145782
Name: Monetary, dtype: float64

In [13]:
# count of each segment
rfm['Segment'].value_counts()

Segment
At risk        1190
Hibernating    1138
Loyalists      1028
Champions       982
Name: count, dtype: int64

In [14]:
# market basket analysis 
query_tran = " SELECT invoice_no, stock_id FROM raw_transactions; "

In [15]:
basket_df = pd.read_sql(query_tran, engine)

In [16]:
basket_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397884 entries, 0 to 397883
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   invoice_no  397884 non-null  object
 1   stock_id    397884 non-null  object
dtypes: object(2)
memory usage: 6.1+ MB


In [17]:
basket_df.head()

,invoice_no,stock_id
0,536365,85123A
1,536365,71053
2,536365,84406B
3,536365,84029G
4,536365,84029E


In [18]:
basket = (basket_df.groupby(['invoice_no', 'stock_id'])['stock_id'].count().unstack().fillna(0))

In [19]:
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

C:\Users\ASHISH\AppData\Local\Temp\ipykernel_21376\3498954818.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)


In [20]:
frequent_itemsets = apriori(basket,min_support=0.02,use_colnames=True)

C:\Users\ASHISH\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:175: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [21]:
rules = association_rules(frequent_itemsets,metric="lift",min_threshold=1)

In [22]:
rules = rules.sort_values(by="lift", ascending=False)

In [23]:
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head())

       antecedents     consequents   support  confidence       lift
86         (22698)  (22697, 22699)  0.021045    0.701439  24.027846
83  (22697, 22699)         (22698)  0.021045    0.720887  24.027846
84  (22698, 22699)         (22697)  0.021045    0.894495  23.989564
85         (22697)  (22698, 22699)  0.021045    0.564399  23.989564
57         (22698)         (22697)  0.024822    0.827338  22.188466


In [24]:
rfm.to_csv("rfm_customer_segments.csv", index=False)
rules.to_csv("rules.csv", index=False)
